#### World Population Prospects 2019
#### File POP/1-1: Total population (both sexes combined) by region, subregion and country, ####annually for 1950-2100 (thousands)
#### Constant-mortality variant, 2020 - 2100
#### POP/DB/WPP/Rev.2019/POP/F01-1
#### © August 2019 by United Nations, made available under a Creative Commons license CC BY 3.0 IGO: #### http://creativecommons.org/licenses/by/3.0/igo/
#### United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects 2019, Online Edition. Rev. 1.

In [1]:
import pandas as pd
import requests
import os
import csv
#from googletrans import Translator

In [2]:
pd.set_option("display.precision", 2)

### OMS https://covid19.who.int/WHO-COVID-19-global-data.csv

In [3]:
direccion_url = 'https://covid19.who.int/WHO-COVID-19-global-data.csv'

In [4]:
'''Uncomment para definir el Path'''

direccion_local = '/Users/Dr.LuisEvelioRestrepoGarcia/datosRecopilador/CovidMundo.csv'    # En mi Apple
'''Obtiene la base de datos actualizada y la guardar en disco local.'''

'Obtiene la base de datos actualizada y la guardar en disco local.'

In [5]:
datos_req = requests.get(direccion_url)
try:
    datos_req.raise_for_status()
except Exception as exc:
    print('Problema en la descarga: %s' % (exc))
print(datos_req.status_code == requests.codes.ok)
playFile = open('CovidMundo.csv', 'wb')
for chunk in datos_req.iter_content(100000):
    playFile.write(chunk)
playFile.close()

True


In [6]:
covMun = pd.read_csv('CovidMundo.csv', usecols=['Date_reported', 'Country', 
       'New_cases', 'Cumulative_cases', 'New_deaths', 'Cumulative_deaths'])

In [7]:
covMun.columns

Index(['Date_reported', 'Country', 'New_cases', 'Cumulative_cases',
       'New_deaths', 'Cumulative_deaths'],
      dtype='object')

In [8]:
covMun.rename({'Date_reported': 'Fecha_reporte', 'Country': 'País', 'New_cases':'Total_Casos', 
                'Cumulative_cases': 'Casos_acumulados', 'New_deaths':'Fallecidos', 
                'Cumulative_deaths':'Muertes_acumuladas'}, axis='columns', inplace=True)
covMun = covMun.groupby('País')[['Total_Casos','Fallecidos']].sum()


covMun = covMun.reset_index()

covMun

,País,Total_Casos,Fallecidos
0,Afghanistan,48540,1921
1,Albania,44436,936
2,Algeria,89416,2539
3,American Samoa,0,0
4,Andorra,7127,78
...,...,...,...
230,Wallis and Futuna,3,0
231,Yemen,2083,607
232,Zambia,17931,364
233,Zimbabwe,10839,294


In [9]:
type(covMun.loc[0,'Fallecidos'])

numpy.int64

translator = Translator()
covMun['Spanish'] = covMun['País'].apply(translator.translate, src='en', dest='es').apply(getattr, args=('text',))
covMun.to_csv('CovidMundoT.csv')

covMun.drop('País', axis= 'columns', inplace=True)
covMun.rename({'Spanish':'País'}, axis = 'columns', inplace=True)

In [10]:
covMun.set_index('País', inplace=True)

In [11]:
mundo= pd.read_csv('/Users/Dr.LuisEvelioRestrepoGarcia/REPOS/poblacionMundial.csv')
mundo.rename({'Spanish':'País'}, axis='columns', inplace=True)
mundo.set_index('País', inplace=True)
mundo = mundo['2020']

In [12]:
Tasas_mundiales = pd.merge(covMun, mundo, left_index=True, right_index=True)

Tasas_mundiales.rename({'2020':'Población'}, axis ='columns', inplace=True)

Tasas_mundiales['Tasa de mortalidad por millón de habitantes']= (Tasas_mundiales['Fallecidos']/Tasas_mundiales['Población'])*1000000
Tasas_mundiales['Tasa de letalidad por cien habitantes']= (Tasas_mundiales['Fallecidos']/Tasas_mundiales['Total_Casos'])*100

In [13]:
Tasas_mundiales.dtypes

Total_Casos                                      int64
Fallecidos                                       int64
Población                                        int64
Tasa de mortalidad por millón de habitantes    float64
Tasa de letalidad por cien habitantes          float64
dtype: object

In [14]:
pais = ['Colombia']

In [15]:
pais

['Colombia']

In [16]:
Tasas_mundiales.loc[pais].sort_values('Tasa de mortalidad por millón de habitantes')

,Total_Casos,Fallecidos,Población,Tasa de mortalidad por millón de habitantes,Tasa de letalidad por cien habitantes
País,,,,,
Colombia,1384610,38158,50883000,749.92,2.76


In [17]:
Tasas_mundiales.to_csv('Tasas_mundiales.csv')